In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# !pip install evaluate
# 
from sklearn.model_selection import train_test_split
import csv
from optuna import Trial
from typing import Dict, Union, Any
import whatthepatch
import evaluate

In [2]:
df = pd.read_csv(r'/root/autodl-tmp/CommitFit/dataset/dataset.csv', index_col=0, encoding='utf_8_sig')
df.fillna('', inplace=True)
label2id={'negative':0,'positive':1}
df = df.replace({"label": label2id})
df

/tmp/ipykernel_17220/3052316322.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"label": label2id})


,label,message,diff
0,0,Merge pull request #46 from rufferson/saslx-tl...,diff --git a/lib/DJabberd.pm b/lib/DJabberd.pm...
1,1,Fix leaks in kadmin server stubs [CVE-2015-863...,diff --git a/src/kadmin/server/server_stubs.c ...
2,1,Validate authorization request on approval\n\n...,diff --git a/spring-security-oauth2/src/main/j...
3,1,Release 2.72.4+171110,diff --git a/application/config/version.php b/...
4,0,Fixing compiler warnings.,diff --git a/src/main.c b/src/main.c\nindex 50...
...,...,...,...
10107,0,les: remove useless protocol defines (#22115)\...,diff --git a/les/benchmark.go b/les/benchmark....
10108,1,Merge pull request #2067 from realm/tg-swift-l...,diff --git a/CHANGELOG.md b/CHANGELOG.md\ninde...
10109,0,[fix] 新規アカウント作成ページの翻訳を修正,diff --git a/app/locales/ja_JP/LC_MESSAGES/mes...
10110,0,Merge branch 'hotfixes',diff --git a/CHANGELOG.txt b/CHANGELOG.txt\nin...


In [1]:
train, test = train_test_split(df,train_size=0.7,stratify=df['label'],random_state=1)

NameError: name 'train_test_split' is not defined

In [2]:
train

NameError: name 'train' is not defined

In [7]:
len(train)

7078

In [8]:
train['label'].value_counts()

label
0    4443
1    2635
Name: count, dtype: int64

In [9]:
test['label'].value_counts()

label
0    1904
1    1130
Name: count, dtype: int64

In [10]:
from sklearn import  metrics

# def compute_metrics(y_pred, y_test):
#     # print(y_pred,y_test)
#     # classification_report = metrics.classification_report(y_test,y_pred)
#     accuracy_score = metrics.accuracy_score(y_test,y_pred)
#     recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
#     f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

#     # return {"classification_report": classification_report}
#     return {"accuracy": accuracy_score, "recall": recall_score, "f1":f1_score}

In [11]:
train

,label,message,diff
1863,1,XSS 취약점 수정,diff --git a/adm/board_form_update.php b/adm/b...
8529,0,"Auto merge of #4754 - daira:fix-enable-debug, ...",diff --git a/depends/hosts/darwin.mk b/depends...
902,1,CAMEL-9309: Make it easier to turn on|off java...,diff --git a/components/camel-ahc/src/main/jav...
6639,1,Fixed bug #72627: Memory Leakage In exif_proce...,diff --git a/ext/exif/exif.c b/ext/exif/exif.c...
6912,0,Merge branch 'dev',diff --git a/.htaccess b/.htaccess\nindex 3293...
...,...,...,...
1639,1,Restrict permissions on cinder configuration\n...,diff --git a/manifests/base.pp b/manifests/bas...
5572,1,avformat/avidec: Limit formats in gab2 to srt ...,diff --git a/libavformat/avidec.c b/libavforma...
7610,1,Fixed possible direct calls,diff --git a/phpmyfaq/admin/stat.ratings.php b...
6941,1,"Reject qname's wirelength > 255, `chopOff()` h...",diff --git a/pdns/dnsparser.cc b/pdns/dnsparse...


In [77]:
def find_longest_list(lists):
    # print(lists)
    if not lists:
        return None
    try:
        longest_list = max(lists, key=len)
    except:
        longest_list = None
    return longest_list

In [78]:
def extract_lines(changes):
    try:
       res = ''.join([x.line for x in changes])
    except:
       res = None
    return res

In [79]:
df['command'] = df['diff'].apply(lambda x : extract_lines(find_longest_list([y.changes for y in whatthepatch.parse_patch(x)])))

NameError: name 'whatthepatch' is not defined

In [53]:
# from transformers import RobertaModel, 
from transformers import AutoModelForSequenceClassification,   TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained('/root/autodl-tmp/models/codebert-base')
tokenizer = AutoTokenizer.from_pretrained('/root/autodl-tmp/models/codebert-base')

Some weights of the model checkpoint at /root/autodl-tmp/models/codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /root/autodl-tmp/models/codebert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model 

In [69]:
training_args = TrainingArguments(
    output_dir="./my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

In [76]:
encoded_train = tokenizer(train['command'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
encoded_test = tokenizer(test['command'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)

In [80]:
class CommitDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [81]:
train_dataset = CommitDataset(encoded_train, list(train['label']))
test_dataset = CommitDataset(encoded_test, list(test['label']))

In [82]:
def compute_metrics(eval_pred):
    accuracy = evaluate.load('/root/autodl-tmp/evaluate/metrics/accuracy')
    precision =  evaluate.load("/root/autodl-tmp/evaluate/metrics/precision")
    recall =  evaluate.load("/root/autodl-tmp/evaluate/metrics/recall")
    f1 =  evaluate.load("/root/autodl-tmp/evaluate/metrics/f1")
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy.compute(predictions=predictions, references=labels)
    precision = precision.compute(predictions=predictions, references=labels, average='weighted')
    recall = recall.compute(predictions=predictions, references=labels,average='weighted')
    f1 = f1.compute(predictions=predictions, references=labels,average='weighted')
    # accuracy_score = metrics.accuracy_score(labels,predictions)
    # precision_score = metrics.precision_score(labels,predictions, average='weighted')
    # recall_score = metrics.recall_score(labels,predictions,average='weighted')
    # f1_score = metrics.f1_score(labels,predictions,average='weighted')

    # return {"classification_report": classification_report}
    # return {"precision": precision_score,"recall": recall_score, "f1":f1_score, "accuracy": accuracy_score}
    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [83]:
# !export https_proxy=http://turbo.gpushare.com:30000 http_proxy=http://turbo.gpushare.com:30000
# !export https_proxy=http://turbo2.gpushare.com:30000 http_proxy=http://turbo2.gpushare.com:30000

In [84]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/tmp/ipykernel_17220/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.389213,{'precision': 0.8287570879645517},{'recall': 0.8292682926829268},{'f1': 0.8256368463229387},{'accuracy': 0.8292682926829268}
2,0.432000,0.385219,{'precision': 0.8498861061486519},{'recall': 0.8437705998681608},{'f1': 0.8452585293845021},{'accuracy': 0.8437705998681608}
3,0.289800,0.503729,{'precision': 0.8418694501719088},{'recall': 0.8325642715886619},{'f1': 0.834524856377495},{'accuracy': 0.8325642715886619}
4,0.178600,0.689953,{'precision': 0.8502693250360583},{'recall': 0.8503625576796309},{'f1': 0.8476383817637505},{'accuracy': 0.8503625576796309}


Trainer is attempting to log a value of "{'precision': 0.8287570879645517}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.8292682926829268}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.8256368463229387}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.8292682926829268}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/tmp/ipykernel_17220/77294352.py:7: UserWarning: To copy construct from a tensor, it is re

RuntimeError: [enforce fail at inline_container.cc:424] . unexpected pos 33984 vs 33888